Interacting Proteins

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as pu

In [2]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)


In [3]:
gene = 'PTEN'

In [4]:
prot = gbm.get_proteomics()
p = list(prot.columns)
len(p)

11141

In [5]:
mut_type_gbm = gbm.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
del_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


Name,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00104,0.077630,0.487228,-0.254208,-0.144373,0.551881,-0.025276,-0.467451,-0.089511,-0.078806,0.329603,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Deletion
C3L-00365,-0.145975,0.798796,0.184242,-0.470603,NaN,0.390211,0.245466,-0.609998,0.118625,-0.086927,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Deletion
C3L-00674,0.821991,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Deletion
C3L-00677,-0.064567,0.129385,0.047751,-0.118187,0.237434,NaN,0.303847,0.322163,-0.555479,-0.363414,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Deletion
C3L-01040,-0.763691,-1.031834,-0.217194,-0.695701,0.184173,-0.474816,-0.051789,0.344842,-0.642746,0.068863,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Deletion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03183,0.230843,0.426624,0.420155,0.303960,0.078341,0.833128,0.205727,-0.434020,0.151202,0.208972,...,-0.021466,-0.049569,0.128461,-0.437797,-0.144343,-0.024981,-0.275897,-0.116367,0.351479,Deletion
C3N-03184,-0.635316,-0.478657,0.647004,-0.050322,-0.440517,0.092256,-0.006924,-0.511331,0.101877,0.128163,...,-0.034616,0.158269,NaN,NaN,0.906358,0.287317,0.745880,-0.320613,-0.180511,Deletion
C3N-03186,0.616640,0.767029,-0.231897,0.050841,0.304359,-0.384608,-0.205284,0.513575,0.151769,-0.307037,...,-0.110846,-0.069605,-0.183921,NaN,-0.681170,0.095601,-0.096716,0.049811,-0.409252,Deletion


In [6]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
g_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if g_sig is not None:
    g_sig_list = list(g_sig.Comparison)
else: 
    g_sig_list = None
print('significant pvals: \n',g_sig)

# Get all pvals
g = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
g_pval = g.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
#g_pval = g_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
g_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
               Comparison       P_Value
0       ARMH3_proteomics  5.396032e-11
1        CUTC_proteomics  1.593480e-10
2     PIP4K2A_proteomics  1.009419e-09
3        CUL2_proteomics  1.122076e-09
4        GDI2_proteomics  1.302273e-09
...                  ...           ...
1882  GOLGA7B_proteomics  8.446816e-03
1883     TEX2_proteomics  8.452685e-03
1884     MAFF_proteomics  8.463737e-03
1885   LRPAP1_proteomics  8.466030e-03
1886  PPP2R2B_proteomics  8.466960e-03

[1887 rows x 2 columns]


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,Gbm_P_Value
0,ARMH3_proteomics,5.396032e-11
1,CUTC_proteomics,1.593480e-10
2,PIP4K2A_proteomics,1.009419e-09
3,CUL2_proteomics,1.122076e-09
4,GDI2_proteomics,1.302273e-09
...,...,...
11136,ZNF805_proteomics,NaN
11137,ZNF813_proteomics,NaN
11138,ZNF814_proteomics,NaN
11139,ZNF888_proteomics,NaN


In [7]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

gbm_d = {}


for prot in p:
    dif_gbm = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    gbm_d[prot+'_proteomics'] = dif_gbm
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif_gbm)

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm_Median'])
gbm_df = gbm_df.reset_index().rename(columns={'index':'Proteomics'})

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [8]:
#TESTS
gbm_df.loc[gbm_df['Proteomics'] == 'MAFF_proteomics']

print(wt_med.MAFF_proteomics, '-', del_med.MAFF_proteomics)

0.005661958017430049 - 0.20855503331380604


In [9]:
g_merged = g_pval.merge(gbm_df, on='Proteomics',how='inner')
g_merged

,Proteomics,Gbm_P_Value,Gbm_Median
0,ARMH3_proteomics,5.396032e-11,0.405134
1,CUTC_proteomics,1.593480e-10,0.553255
2,PIP4K2A_proteomics,1.009419e-09,0.838882
3,CUL2_proteomics,1.122076e-09,0.586396
4,GDI2_proteomics,1.302273e-09,0.610188
...,...,...,...
11136,ZNF805_proteomics,NaN,NaN
11137,ZNF813_proteomics,NaN,NaN
11138,ZNF814_proteomics,NaN,0.265936
11139,ZNF888_proteomics,NaN,NaN


Endo

In [10]:
#Endo ttest

prot = endo.get_proteomics()
p = list(prot.columns)
print('# all proteins:',len(p))

mut_type = endo.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt['Mutation'].value_counts()
trunc_wt

# all proteins: 10999


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 11)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Name,A1BG_proteomics,A2M_proteomics,A2ML1_proteomics,A4GALT_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,...,ZSWIM9_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,-1.180,-0.8630,-0.8020,0.2220,0.2560,0.6650,1.2800,-0.3390,0.4120,-0.6640,...,NaN,0.02290,0.109,NaN,-0.332,-0.4330,-1.020,-0.12300,-0.08590,Truncation
C3L-00032,-0.528,-1.3200,0.4350,NaN,-0.2400,1.0400,-0.0213,-0.0479,0.4190,-0.5000,...,-0.14500,0.01050,-0.116,NaN,0.151,-0.0740,-0.540,0.32000,-0.41900,Truncation
C3L-00098,-0.374,-0.0206,-0.5370,0.3110,0.3750,0.0131,-1.1000,NaN,0.5650,-0.1010,...,NaN,-1.22000,-0.562,0.9370,-0.646,0.2070,-1.850,-0.17600,0.05130,Wildtype_Tumor
C3L-00137,-1.320,-0.7080,-0.8080,-0.0709,0.1380,0.6560,-0.2800,-0.1280,0.2170,-0.1330,...,-0.00491,0.18000,0.513,0.5490,-0.681,-0.2850,-0.564,-0.08760,0.00698,Truncation
C3L-00139,-0.467,0.3700,-0.3390,NaN,0.4340,0.0358,-0.1750,0.1810,0.1160,-0.0512,...,0.23900,0.14000,1.070,0.6070,0.486,0.1690,-0.632,-0.20300,-0.06850,Wildtype_Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01520,-1.070,-0.7120,0.0462,-0.0471,0.2270,1.3500,1.2100,0.0048,0.6820,-0.2240,...,-0.40100,0.55700,0.827,0.3480,-0.187,-0.1070,-0.830,0.06200,-0.52800,Truncation
C3N-01521,-1.280,-0.7360,-0.5520,0.1140,0.2730,1.0700,0.8500,NaN,-0.5360,-0.2660,...,NaN,-0.00364,0.763,0.0784,-0.203,0.0132,-1.200,-0.35700,0.02990,Wildtype_Tumor
C3N-01537,-0.290,-0.3200,3.1700,-0.9070,0.0317,-0.0425,NaN,-0.2730,-0.0747,0.1240,...,-0.52100,0.05230,0.515,0.5850,0.138,-0.1900,-0.966,-0.00627,-0.24900,Wildtype_Tumor


In [11]:
cols = list(trunc_wt.columns[:-1])

# Get only sig sites
e_sig = u.wrap_ttest(trunc_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if e_sig is not None:
    e_sig_list = list(e_sig.Comparison)
else: 
    e_sig_list = None
print('significant pvals: \n',e_sig)

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})
e_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
             Comparison       P_Value
0     NOL10_proteomics  8.999599e-08
1      ABT1_proteomics  3.141336e-07
2     UTP25_proteomics  4.308134e-07
3    TOPBP1_proteomics  4.885533e-07
4      RIF1_proteomics  9.004982e-07
..                 ...           ...
397   ACTR5_proteomics  1.788167e-03
398   RSPH3_proteomics  1.789386e-03
399   COPS5_proteomics  1.793000e-03
400   DDAH1_proteomics  1.822621e-03
401   UCHL1_proteomics  1.825674e-03

[402 rows x 2 columns]


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,En_P_Value
0,NOL10_proteomics,8.999599e-08
1,ABT1_proteomics,3.141336e-07
2,UTP25_proteomics,4.308134e-07
3,TOPBP1_proteomics,4.885533e-07
4,RIF1_proteomics,9.004982e-07
...,...,...
10994,ZNF586_proteomics,NaN
10995,ZNF630_proteomics,NaN
10996,ZNF772_proteomics,NaN
10997,ZNF79_proteomics,NaN


In [12]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - trunc_med[prot+'_proteomics']
    en_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', trunc_med[prot+'_proteomics'], '=',dif)

en_df = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_df = en_df.reset_index().rename(columns={'index':'Proteomics'})
en_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,En_Median
0,A1BG_proteomics,0.06100
1,A2M_proteomics,-0.07400
2,A2ML1_proteomics,-0.22300
3,A4GALT_proteomics,-0.03995
4,AAAS_proteomics,0.19200
...,...,...
10994,ZXDC_proteomics,0.42800
10995,ZYG11B_proteomics,0.19420
10996,ZYX_proteomics,-0.21600
10997,ZZEF1_proteomics,-0.05320


In [13]:
#TESTS
I = 'DOCK1'
en_df.loc[en_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', trunc_med[I+'_proteomics'])

print(e_pval.loc[e_pval['Proteomics'] == I+'_proteomics'])

-0.16699999999999998 - -0.284
            Proteomics  En_P_Value
4777  DOCK1_proteomics    0.253937


In [14]:
e_merged = e_pval.merge(en_df, on='Proteomics',how='inner')
e_merged

,Proteomics,En_P_Value,En_Median
0,NOL10_proteomics,8.999599e-08,0.44200
1,ABT1_proteomics,3.141336e-07,0.48510
2,UTP25_proteomics,4.308134e-07,0.73450
3,TOPBP1_proteomics,4.885533e-07,0.61795
4,RIF1_proteomics,9.004982e-07,0.54640
...,...,...,...
10994,ZNF586_proteomics,NaN,NaN
10995,ZNF630_proteomics,NaN,-0.99250
10996,ZNF772_proteomics,NaN,NaN
10997,ZNF79_proteomics,NaN,NaN


Ovarian

In [15]:
o = cptac.Ovarian()

In [16]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
gene = 'PTEN'

prot = o.get_proteomics()
prot = o.reduce_multiindex(prot, levels_to_drop = 1)
p = list(prot.columns)
len(p)

cptac warning: Due to dropping the specified levels, dataframe now has 1162 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


10686

In [17]:
mut_type_gbm = o.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = o.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = o.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
del_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: Due to dropping the specified levels, dataframe now has 1162 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packa

Name,A1BG_proteomics,A2M_proteomics,A2ML1_proteomics,AAAS_proteomics,AACS_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,AAR2_proteomics,...,ZSCAN32_proteomics,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01OV007,0.133634,0.765120,-0.778849,0.118236,0.112740,0.054942,0.313055,-0.786270,-0.384680,0.064017,...,-0.323516,0.074209,0.232999,-0.195793,NaN,-0.145053,0.590217,0.019414,NaN,Wildtype_Tumor
01OV017,-0.432786,-0.461906,-0.533497,0.041800,0.604114,0.351294,0.546605,0.603935,-0.235374,0.059084,...,0.032004,0.016276,0.587911,0.574339,NaN,0.075401,-0.437564,-0.208831,NaN,Deletion
01OV018,-0.013943,0.065842,NaN,-0.010514,0.541935,0.169826,-0.023032,-0.057549,-0.397627,0.199581,...,0.002237,0.206993,0.111037,1.020321,NaN,-0.245864,0.008411,0.130504,-0.367433,Wildtype_Tumor
01OV023,1.234203,1.512082,-0.703050,0.181152,0.087719,0.226861,-0.100766,-0.516022,-0.456512,-0.257629,...,NaN,-0.529948,0.034713,0.149523,-0.313370,-0.507473,0.221818,0.278313,-1.008998,Deletion
01OV026,-0.482872,0.343872,-1.150936,0.315155,0.097235,-0.160208,-0.125165,-0.768776,-0.372368,0.450086,...,-0.752819,-0.181027,0.179612,0.007854,-0.106049,0.377063,-0.295592,0.011997,-0.236003,Wildtype_Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26OV002,1.942785,2.028395,-0.506808,-0.237500,-0.767353,-0.094263,0.164654,-0.120120,-0.560682,0.116122,...,0.291665,-0.205511,0.280838,1.143053,0.545983,0.045314,-0.463719,-0.321074,0.103304,Deletion
26OV008,-0.576189,-1.723434,2.023556,0.066470,0.576987,0.725690,0.610382,-0.704039,0.121085,0.534117,...,0.021077,0.054952,0.247094,0.365626,NaN,0.101908,0.338000,-0.429887,-0.578476,Wildtype_Tumor
26OV009,0.072513,0.594745,0.609137,0.584056,0.041583,-0.269243,-0.461148,-0.297156,-0.027213,0.596118,...,-0.578555,0.533873,0.440283,0.738610,NaN,0.482615,-0.685922,-0.286718,-0.542423,Wildtype_Tumor


In [18]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
o_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if o_sig is not None:
    o_sig_list = list(o_sig.Comparison)
else: 
    o_sig_list = None
print('significant pvals: \n',o_sig)

# Get all pvals
o = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
o_pval = o.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Ov_P_Value'})
#o_pval = o_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
o_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
          Comparison       P_Value
0  MMS19_proteomics  1.058914e-08


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Proteomics,Ov_P_Value
0,MMS19_proteomics,1.058914e-08
1,RACK1_proteomics,8.573522e-05
2,PI4K2A_proteomics,9.310383e-05
3,WAPL_proteomics,1.724680e-04
4,IDE_proteomics,2.011475e-04
...,...,...
10072,SAXO2_proteomics,NaN
10073,SH3GL3_proteomics,NaN
10074,SLC22A3_proteomics,NaN
10075,SSX2B_proteomics,NaN


In [19]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

ov_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    ov_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

o_df = pd.DataFrame.from_dict(ov_d, orient='index', columns=['Ov_Median'])
o_df = o_df.reset_index().rename(columns={'index':'Proteomics'})
#o_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [20]:
#TESTS
I = 'MAFF'
o_df.loc[o_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', del_med[I+'_proteomics'])

print(o_pval.loc[o_pval['Proteomics'] == I+'_proteomics'])

-0.6560350245 - -0.534134515
           Proteomics  Ov_P_Value
9772  MAFF_proteomics    0.968044


In [21]:
o_merged = o_pval.merge(o_df, on='Proteomics',how='inner')
o_merged

,Proteomics,Ov_P_Value,Ov_Median
0,MMS19_proteomics,1.058914e-08,0.324897
1,RACK1_proteomics,8.573522e-05,0.191250
2,PI4K2A_proteomics,9.310383e-05,0.298399
3,WAPL_proteomics,1.724680e-04,0.218775
4,IDE_proteomics,2.011475e-04,0.208025
...,...,...,...
10072,SAXO2_proteomics,NaN,0.099993
10073,SH3GL3_proteomics,NaN,-0.843461
10074,SLC22A3_proteomics,NaN,-0.783325
10075,SSX2B_proteomics,NaN,NaN


Breast

In [22]:
#b = cptac.Brca()

In [23]:
prot = b.get_proteomics()
prot = b.reduce_multiindex(prot, levels_to_drop = 1)
p = list(prot.columns)
len(p)

mut_type = b.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 10)
cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 11)

Wildtype_Tumor    80
Deletion          24
Name: Mutation, dtype: int64

In [24]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
b_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if b_sig is not None:
    b_sig_list = list(b_sig.Comparison)
else: 
    b_sig_list = None
print('significant pvals: \n',b_sig)

# Get all pvals
b_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
b_pval = b_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Brca_P_Value'})
#b_pval = b_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#b_pval

significant pvals: 
         Comparison   P_Value
0  PTEN_proteomics  0.000004


In [25]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

b_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    b_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

b_df = pd.DataFrame.from_dict(b_d, orient='index', columns=['Brca_Median'])
b_df = b_df.reset_index().rename(columns={'index':'Proteomics'})
b_df

,Proteomics,Brca_Median
0,A1BG_proteomics,1.00440
1,A2M_proteomics,0.80760
2,A2ML1_proteomics,-1.31745
3,AAAS_proteomics,-0.34660
4,AACS_proteomics,-0.73635
...,...,...
9766,ZXDC_proteomics,0.24385
9767,ZYG11B_proteomics,-0.11285
9768,ZYX_proteomics,-0.28070
9769,ZZEF1_proteomics,0.18565


In [26]:
#TESTS
I = 'DOCK1'
b_df.loc[b_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', del_med[I+'_proteomics'])

b_sig.iloc[350:]
print(b_pval.loc[b_pval['Proteomics'] == I+'_proteomics'])

0.32399999999999995 - -0.016050000000000002
            Proteomics  Brca_P_Value
1119  DOCK1_proteomics      0.056169


In [27]:
b_merged = b_pval.merge(b_df, on='Proteomics',how='inner')
b_merged

,Proteomics,Brca_P_Value,Brca_Median
0,PTEN_proteomics,0.000004,1.00800
1,EIF4H_proteomics,0.000025,-0.41295
2,MIEF1_proteomics,0.000031,-0.84135
3,TMSB10_proteomics,0.000043,-0.87630
4,DRG1_proteomics,0.000065,-0.75135
...,...,...,...
9766,POLR3E_proteomics,0.999356,0.09685
9767,IGFBP7_proteomics,0.999595,-0.01140
9768,AAMP_proteomics,0.999613,-0.15430
9769,TRA2B_proteomics,0.999771,0.17180


Colon

In [28]:
#col = cptac.Colon()

In [29]:
prot = col.get_proteomics()
p = list(prot.columns)
len(p)

mut_type = col.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = col.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 100 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 201 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05

Wildtype_Tumor    76
Deletion          25
Name: Mutation, dtype: int64

In [30]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
c_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if c_sig is not None:
    c_sig_list = list(c_sig.Comparison)
else: 
    c_sig_list = None
print('significant pvals: \n',c_sig)

# Get all pvals
c_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
c_pval = c_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Colon_P_Value'})
#c_pval = c_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#c_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
 None


In [31]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

c_df = pd.DataFrame.from_dict(d, orient='index', columns=['Colon_Median'])
c_df = c_df.reset_index().rename(columns={'index':'Proteomics'})
#c_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [32]:
#TESTS
I = 'MAFF'
c_df.loc[c_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', del_med[I+'_proteomics'])

print(c_pval.loc[c_pval['Proteomics'] == I+'_proteomics'])

0.19399999999999998 - -0.1295
           Proteomics  Colon_P_Value
1757  MAFF_proteomics       0.200121


In [33]:
c_merged = c_pval.merge(c_df, on='Proteomics',how='inner')
c_merged

,Proteomics,Colon_P_Value,Colon_Median
0,DFFA_proteomics,0.000037,0.19930
1,WAPL_proteomics,0.000097,0.19605
2,SEC14L2_proteomics,0.000136,0.66800
3,GBF1_proteomics,0.000164,0.12630
4,STK11IP_proteomics,0.000239,0.31250
...,...,...,...
8062,ZNF841_proteomics,NaN,-0.78250
8063,ZNF888_proteomics,NaN,NaN
8064,ZNHIT6_proteomics,NaN,NaN
8065,ZNRD1_proteomics,NaN,0.06800


Head and Neck

In [34]:
#h = cptac.Hnscc()

In [35]:
prot = h.get_proteomics()
p = list(prot.columns)
len(p)

mut_type = h.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
#prot_and_mutations = h.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 109 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-02617, C3N-02727 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 176 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)


Wildtype_Tumor    87
Deletion          18
Name: Mutation, dtype: int64

In [36]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
h_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if h_sig is not None:
    h_sig_list = list(h_sig.Comparison)
else: 
    h_sig_list = None
print('significant pvals: \n',h_sig)

# Get all pvals

h_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
h_pval = h_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Hnscc_P_Value'})
#h_pval = h_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#h_pval


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
              Comparison       P_Value
0        NLN_proteomics  1.113430e-09
1     AKR1C3_proteomics  1.032977e-08
2     AKR1C1_proteomics  6.931627e-08
3       ADI1_proteomics  1.601880e-07
4      EPHX1_proteomics  3.899394e-07
..                  ...           ...
523    FLAD1_proteomics  2.232897e-03
524    GSTM4_proteomics  2.234528e-03
525   ELOVL5_proteomics  2.239241e-03
526  SUPT16H_proteomics  2.241138e-03
527  ANKRD28_proteomics  2.243254e-03

[528 rows x 2 columns]


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

In [37]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

h_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    h_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

h_df = pd.DataFrame.from_dict(h_d, orient='index', columns=['Hnscc_Median'])
h_df = h_df.reset_index().rename(columns={'index':'Proteomics'})
#h_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [38]:
#TESTS
I = 'DOCK1'
h_df.loc[h_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', del_med[I+'_proteomics'])

print(h_pval.loc[h_pval['Proteomics'] == I+'_proteomics'])

25.186277574373303 - 25.0734306518224
            Proteomics  Hnscc_P_Value
5122  DOCK1_proteomics       0.259882


In [39]:
h_merged = h_pval.merge(h_df, on='Proteomics',how='inner')
#del_wt[['PREX2_proteomics','Mutation']].dropna() # in median, not in pval (only 1 value for deletion)
h_merged

,Proteomics,Hnscc_P_Value,Hnscc_Median
0,NLN_proteomics,1.113430e-09,-0.431666
1,AKR1C3_proteomics,1.032977e-08,-0.951905
2,AKR1C1_proteomics,6.931627e-08,-1.961262
3,ADI1_proteomics,1.601880e-07,-0.513198
4,EPHX1_proteomics,3.899394e-07,-0.871688
...,...,...,...
11739,ZNF664_proteomics,NaN,NaN
11740,ZNF836_proteomics,NaN,NaN
11741,ZNF862_proteomics,NaN,NaN
11742,ZSCAN31_proteomics,NaN,NaN


Lung

In [40]:
#l = cptac.Luad()

In [41]:
prot = l.get_proteomics()
prot = l.reduce_multiindex(prot, levels_to_drop = 1, flatten = True)
p = list(prot.columns)
len(p)

mut_type = l.get_genotype_all_vars(gene)
mut_type = mut_type.rename(columns={'PTEN':'cnv'})

# different code because no somatic mutation data for pten (can't join to somatic mutations)
omics = l.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1="PTEN", 
    genes2=p)
omics = l.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
omics = omics.drop(columns='PTEN_CNV')

# Get only tumor samples
p = l.get_proteomics(tissue_type='tumor')
tumor_ids = list(p.index)
get = omics.index.isin(tumor_ids)
omics = omics[get]

merged = omics.join(mut_type) # checked and there is 110 tumor samples for lung

compare = ['No_Mutation','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
print(del_wt['Mutation'].value_counts())
del_wt = del_wt.dropna(axis='columns', how='all')
del_wt = del_wt.loc[:,~del_wt.columns.duplicated()] # SH1, PRK3R1 both have double columns

cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)
cptac warning: You tried to flatten an index that didn't have multiple levels, so we didn't actually change anything. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00001.N, C3L-00009.N, C3L-00080.N, C3L-00083.N, C3L-00093.N, C3L-00094.N, C3L-00095.N, C3L-00140.N, C3L-00144.N, C3L-00263.N, C3L-00279.N, C3L-00368.N, C3L-00412.N, C3L-00422.N, C3L-00510.N, C3L-00604.N, C3L-00893.N, C3L-00913.N, C3L-00973.N, C3L-01330.N, C3L-01632.N, C3L-01682.N, C3L-01683.N, C3L-01889.N, C3L-01890, C3L-01890.N, C3L-01924.N, C3L-02219.N, C3L-02345.N, C3L-02348.N, C3L-02350.N, C3L-02365.N, C3L-02508.N, C3L-02549.N, C3N-00167.N

No_Mutation    88
Deletion       17
Name: Mutation, dtype: int64


cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 12)
cptac warning: You tried to flatten an index that didn't have multiple levels, so we didn't actually change anything. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 12)


In [42]:
cols = list(del_wt.columns[:-2])

# Get only sig sites
l_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if l_sig is not None:
    l_sig_list = list(l_sig.Comparison)
else: 
    l_sig_list = None
print('significant pvals: \n',l_sig)

# Get all pvals
l_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
l_pval = l_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Luad_P_Value'})
#l_pval = l_pval.replace(to_replace = r'_NP_.*$', value = '', regex = True)
l_pval # isoforms for some proteins

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
               Comparison       P_Value
0        ATE1_proteomics  2.851800e-07
1       SHOC2_proteomics  6.424816e-07
2        EGFR_proteomics  9.121167e-07
3      LANCL2_proteomics  1.742749e-06
4        GGCT_proteomics  2.854181e-06
..                   ...           ...
103    FERMT3_proteomics  4.804371e-04
104  ARHGAP15_proteomics  4.875693e-04
105    CRYBG2_proteomics  5.063478e-04
106     GNAI2_proteomics  5.076782e-04
107     CIAO1_proteomics  5.181887e-04

[108 rows x 2 columns]


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,Luad_P_Value
0,ATE1_proteomics,2.851800e-07
1,SHOC2_proteomics,6.424816e-07
2,EGFR_proteomics,9.121167e-07
3,LANCL2_proteomics,1.742749e-06
4,GGCT_proteomics,2.854181e-06
...,...,...
10311,COG6_proteomics,9.997105e-01
10312,PLD1_proteomics,9.997256e-01
10313,CHMP6_proteomics,9.997659e-01
10314,FHL3_proteomics,9.999702e-01


In [43]:
#Isoform code - fix later
#del_wt.replace(to_replace = r'_NP_.*$', value = '', regex = True)

In [44]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "No_Mutation"]
del_med = d.median()
wt_med = wt.median()

l_d = {}

for prot in cols:
    dif = wt_med[prot] - del_med[prot]
    l_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

l_df = pd.DataFrame.from_dict(l_d, orient='index', columns=['Luad_Median'])
l_df = l_df.reset_index().rename(columns={'index':'Proteomics'})
l_df

,Proteomics,Luad_Median
0,A1BG_proteomics,0.11030
1,A2M_proteomics,0.39840
2,AAAS_proteomics,0.02265
3,AACS_proteomics,0.03800
4,AADAC_proteomics,0.94550
...,...,...
10311,ZXDC_proteomics,-0.20540
10312,ZYG11B_proteomics,-0.45530
10313,ZYX_proteomics,0.00720
10314,ZZEF1_proteomics,0.13655


In [45]:
l_merged = l_pval.merge(l_df, on='Proteomics',how='inner')
#l_merged = l_merged.replace(to_replace = r'_proteomics$', value = '', regex = True)
l_merged

,Proteomics,Luad_P_Value,Luad_Median
0,ATE1_proteomics,2.851800e-07,0.88365
1,SHOC2_proteomics,6.424816e-07,0.60600
2,EGFR_proteomics,9.121167e-07,-2.32970
3,LANCL2_proteomics,1.742749e-06,-1.21475
4,GGCT_proteomics,2.854181e-06,-2.12715
...,...,...,...
10311,COG6_proteomics,9.997105e-01,0.13015
10312,PLD1_proteomics,9.997256e-01,-0.43875
10313,CHMP6_proteomics,9.997659e-01,0.05220
10314,FHL3_proteomics,9.999702e-01,0.06610


In [46]:
#TESTS
I = 'MAFF'
l_df.loc[l_df['Proteomics'] == I+'_proteomics']

print(wt_med[I+'_proteomics'], '-', del_med[I+'_proteomics'])

print(l_pval.loc[l_pval['Proteomics'] == I+'_proteomics'])

0.07719999999999999 - 0.3594
           Proteomics  Luad_P_Value
3257  MAFF_proteomics      0.187607


In [47]:
n = g_merged.merge(h_merged, on='Proteomics',how='outer')
n = n.merge(l_merged, on='Proteomics',how='outer')
n= n.merge(b_merged, on='Proteomics',how='outer')
n = n.merge(o_merged, on='Proteomics',how='outer')
n = n.merge(e_merged, on='Proteomics',how='outer')
all_df = n.merge(c_merged, on='Proteomics',how='outer')
all_df



,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,ARMH3_proteomics,5.396032e-11,0.405134,0.000558,0.135941,0.017908,0.37845,NaN,NaN,NaN,NaN,0.001113,-0.1503,NaN,NaN
1,CUTC_proteomics,1.593480e-10,0.553255,0.000189,0.314307,0.000744,1.01995,0.008295,0.48080,0.060164,0.159424,0.082868,0.2533,0.411506,0.08210
2,PIP4K2A_proteomics,1.009419e-09,0.838882,0.001544,0.224713,0.560996,-0.04535,0.203952,0.07435,0.389187,-0.052955,0.201771,-0.1690,0.199847,-0.10135
3,CUL2_proteomics,1.122076e-09,0.586396,0.400554,-0.017886,0.612583,-0.05695,0.198043,-0.17195,0.786184,-0.033091,0.864381,-0.0232,0.349065,0.07690
4,GDI2_proteomics,1.302273e-09,0.610188,0.630985,-0.001559,NaN,NaN,0.354712,-0.14995,0.569417,0.067389,0.061660,-0.1286,0.990512,0.04450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14360,WTH3DI_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.40100
14361,ZIC4_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00200
14362,ZNF419_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.38150
14363,ZNF705E_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12900


In [48]:
# Keep only genes significant in > 1 cancer
all_sig = []

sig_lists = [g_sig_list, e_sig_list, b_sig_list, o_sig_list, c_sig_list, h_sig_list, l_sig_list]

for next_list in sig_lists:
    if next_list is not None:
        all_sig.append(next_list)
        

flat_list = [item for sublist in all_sig for item in sublist]
sig = set(flat_list) # remove duplicates

bool_df = all_df['Proteomics'].isin(sig)
sig_df = all_df[bool_df]
print(len(sig_df))
sig_df

2584


,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,ARMH3_proteomics,5.396032e-11,0.405134,0.000558,0.135941,0.017908,0.37845,NaN,NaN,NaN,NaN,0.001113,-0.1503,NaN,NaN
1,CUTC_proteomics,1.593480e-10,0.553255,0.000189,0.314307,0.000744,1.01995,0.008295,0.48080,0.060164,0.159424,0.082868,0.2533,0.411506,0.08210
2,PIP4K2A_proteomics,1.009419e-09,0.838882,0.001544,0.224713,0.560996,-0.04535,0.203952,0.07435,0.389187,-0.052955,0.201771,-0.1690,0.199847,-0.10135
3,CUL2_proteomics,1.122076e-09,0.586396,0.400554,-0.017886,0.612583,-0.05695,0.198043,-0.17195,0.786184,-0.033091,0.864381,-0.0232,0.349065,0.07690
4,GDI2_proteomics,1.302273e-09,0.610188,0.630985,-0.001559,NaN,NaN,0.354712,-0.14995,0.569417,0.067389,0.061660,-0.1286,0.990512,0.04450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12981,SUV39H1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,0.169685,-0.37210,NaN,NaN,0.000411,0.7950,NaN,NaN
13236,RSPH3_proteomics,NaN,NaN,NaN,NaN,0.336275,0.81260,NaN,NaN,NaN,NaN,0.001789,-1.5200,NaN,NaN
13251,ARMC3_proteomics,NaN,NaN,NaN,NaN,0.367448,0.57430,NaN,NaN,NaN,NaN,0.000933,-1.0990,NaN,NaN
13275,CCDC57_proteomics,NaN,NaN,NaN,NaN,0.420048,-0.11070,0.621414,0.10775,0.819617,0.087283,0.001480,-0.8445,NaN,NaN


In [49]:
sig_df = sig_df.replace(to_replace = r'_proteomics$', value = '', regex = True)
sig_df = sig_df.set_index('Proteomics')

In [50]:
# check sig_df

sig_df.loc[sig_df.index == 'MAFF']


,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
Proteomics,,,,,,,,,,,,,,
MAFF,0.008464,-0.202893,0.673107,0.060547,0.187607,-0.2822,0.078518,-0.4992,0.968044,-0.121901,0.741379,0.0704,0.200121,0.3235


In [51]:
sig_df.to_csv('sig_pval_all_proteins.csv')

In [52]:
# see sig in cancer
cancer = ['Gbm','En','Brca','Ov','Colon','Hnscc','Luad']
i = 0
for next_list in sig_lists:

    print(cancer[i], ':')
    if next_list is not None: 
        print(len(next_list),'\n')
    #print(next_list, '\n')
    if (i < 6):
        i += 1
    

Gbm :
1887 

En :
402 

Brca :
1 

Ov :
1 

Colon :
Hnscc :
528 

Luad :
108 



# HeatMap df

In [53]:
# Create long df for heat map

cancer = ['Gbm','Hnscc','Luad','Brca','Ov','En','Colon']
merged_dfs = [g_merged,h_merged,l_merged,b_merged,o_merged,e_merged,c_merged]

merged = pd.DataFrame()
i = 0
for c in cancer:
    m = merged_dfs[i]
    m2 = m.assign(Cancer = c)
    m2 = m2.rename(columns={c+'_P_Value': 'P_Value'})
    m2 = m2.rename(columns={c+'_Median': 'Medians'})
    merged = merged.append(m2) 
    if i < 6:
        i += 1

# Keep genes with at least one sig ttest
bool_df2 = merged['Proteomics'].isin(sig)
plot_df = merged[bool_df2]
plot_df = plot_df.replace(to_replace = r'_proteomics$', value = '', regex = True)

# log p-vals for right scale in plot (bigger circle, smaller pval)
plot_df['size'] = plot_df['P_Value'].apply(lambda x: -1*(np.log(x)/10))
plot_df = plot_df.set_index('Proteomics')
plot_df#when df of certain cancer doesnt have gene, not added for that cancer since appending(blank in graph, like nan)

,P_Value,Medians,Cancer,size
Proteomics,,,,
ARMH3,5.396032e-11,0.405134,Gbm,2.364277
CUTC,1.593480e-10,0.553255,Gbm,2.255993
PIP4K2A,1.009419e-09,0.838882,Gbm,2.071389
CUL2,1.122076e-09,0.586396,Gbm,2.060808
GDI2,1.302273e-09,0.610188,Gbm,2.045916
...,...,...,...,...
WWP1,NaN,0.120300,Colon,NaN
ZCCHC9,NaN,-0.406000,Colon,NaN
ZDHHC2,NaN,NaN,Colon,NaN


In [60]:
# CHECK
plot_df.loc[plot_df.index == 'DOCK1']


,P_Value,Medians,Cancer,size
Proteomics,,,,
DOCK1,5.308552e-07,0.444503,Gbm,1.444878
DOCK1,2.598816e-01,0.112847,Hnscc,0.134753
DOCK1,1.314539e-02,0.569100,Luad,0.433168
DOCK1,5.616896e-02,0.340050,Brca,0.287939
DOCK1,3.248402e-03,0.296576,Ov,0.572959
DOCK1,2.539366e-01,0.117000,En,0.137067
DOCK1,7.863180e-02,0.044700,Colon,0.254298


In [55]:
plot_df.to_csv('heat_map_df.csv')

# Keep PosNeg Genes df

In [56]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < -0.3:
            hasNeg = True
        if item > 0.3:
            hasPos = True
           
    if hasPos & hasNeg:
        return True
    return False


In [57]:
sig_df = sig_df.replace(to_replace = r'_proteomics$', value = '', regex = True)


only_med = sig_df.drop(columns= ['Gbm_P_Value','Hnscc_P_Value','Luad_P_Value',
                     'Brca_P_Value','Ov_P_Value','En_P_Value','Colon_P_Value'])
only_med

,Gbm_Median,Hnscc_Median,Luad_Median,Brca_Median,Ov_Median,En_Median,Colon_Median
Proteomics,,,,,,,
ARMH3,0.405134,0.135941,0.37845,NaN,NaN,-0.1503,NaN
CUTC,0.553255,0.314307,1.01995,0.48080,0.159424,0.2533,0.08210
PIP4K2A,0.838882,0.224713,-0.04535,0.07435,-0.052955,-0.1690,-0.10135
CUL2,0.586396,-0.017886,-0.05695,-0.17195,-0.033091,-0.0232,0.07690
GDI2,0.610188,-0.001559,NaN,-0.14995,0.067389,-0.1286,0.04450
...,...,...,...,...,...,...,...
SUV39H1,NaN,NaN,NaN,-0.37210,NaN,0.7950,NaN
RSPH3,NaN,NaN,0.81260,NaN,NaN,-1.5200,NaN
ARMC3,NaN,NaN,0.57430,NaN,NaN,-1.0990,NaN


In [58]:
only_med["Pos_Neg"] = only_med.apply(HasPosNeg, axis = 1)

pn = only_med.loc[only_med['Pos_Neg'] == True]
pn_genes = list(pn.index) # list of genes that have posNeg

get = plot_df.index.isin(pn_genes)
plot_df2 = plot_df[get]

plot_df2['size'] = plot_df2['P_Value'].apply(lambda x: -1*(np.log(x)/10))
plot_df2

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,P_Value,Medians,Cancer,size
Proteomics,,,,
PACS2,1.123526e-08,0.386279,Gbm,1.830421
DOCK5,2.754160e-08,0.862819,Gbm,1.740757
PDZD8,2.315414e-07,0.444260,Gbm,1.527851
HAGH,3.232029e-07,0.601619,Gbm,1.494499
DCTN4,5.256343e-07,0.362140,Gbm,1.445866
...,...,...,...,...
PPDPF,NaN,-0.391000,Colon,NaN
PYHIN1,NaN,1.118000,Colon,NaN
RANGRF,NaN,-0.440000,Colon,NaN


In [63]:
# CHECK
plot_df2.loc[plot_df2.index == 'MAFF']


,P_Value,Medians,Cancer,size
Proteomics,,,,
MAFF,0.008464,-0.202893,Gbm,0.477196
MAFF,0.673107,0.060547,Hnscc,0.039585
MAFF,0.187607,-0.282200,Luad,0.167341
MAFF,0.078518,-0.499200,Brca,0.254443
MAFF,0.968044,-0.121901,Ov,0.003248
MAFF,0.741379,0.070400,En,0.029924
MAFF,0.200121,0.323500,Colon,0.160883


In [59]:
plot_df2.to_csv('pos_neg_df.csv')